We will perfrom various measurements based on all stellar particles within 50 kpc from the subhalo center. To do this, we should construct a data frame that connect the subhalo and the particles from the subhalo centers. This python code construct a Spark Data Frame that joins subhalos and particles within 50 kpc from the subhalo postions and save it in the parquet format.

It takes several hours. I recommend to do this task in .py file and run in the terminal.

# 0. Import packages and set Spark

In [63]:
import numpy as np
import pandas as pd
import glob
import sys
import h5py

from datetime import datetime
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from scipy.spatial import cKDTree

import pyarrow as pa
import pyarrow.parquet as pqw

from functools import reduce
import operator
import gc

h = 0.6774
a = 1/(1+0.62)
box_size = 205000
t_h = 7.786*1e9

In [3]:
from pyspark import SparkContext   
from pyspark.sql import SparkSession

import pyspark.sql.functions as F
from pyspark.sql.functions import broadcast, col, sqrt, pow, floor, monotonically_increasing_id, abs, pmod, least, row_number
import pyspark.sql.types as T
from pyspark import Row
from pyspark.sql.window import Window as W

spark = SparkSession.builder \
    .appName("MyApp") \
    .master("spark://sohnic:7077") \
    .config("spark.executor.memory", "100g")\
    .config("spark.driver.memory", "100g") \
    .getOrCreate()

sc = spark.sparkContext
sc.setCheckpointDir("hdfs://sohnic:54310/tmp/checkpoints")

spark.conf.set("spark.sql.debug.maxToStringFields", 500)
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")
spark.conf.set("spark.sql.hive.filesourcePartitionFileCacheSize", 524288000) # 500MB 

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/25 13:35:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# 1. Read the particle and subhalo data

In [4]:
%%time
# read particle data and save in 'ptldf'
particle_file = 'hdfs://sohnic:54310/data/TNG300/snap99/snap099_cubic_indexed.parquet.snappy'

ptldf = spark.read.option("header","true").option("recursiveFileLookup","true").parquet(particle_file)
ptldf.show(4)

+-----------------+------------------+-----------------+-----------+----------+---------+------------+---+---+---+
|               px|                py|               pz|         vx|        vy|       vz|        mass| ix| iy| iz|
+-----------------+------------------+-----------------+-----------+----------+---------+------------+---+---+---+
|59739.31891187958|18697.943522839945|3046.885422414884| -334.31744|-117.45711|360.27805|7.3169236E-4| 29|  9|  1|
|59803.99389363947| 18650.39813689017|2918.501487478792|-126.714836| 110.34051|122.81836| 5.660612E-4| 29|  9|  1|
|59733.16062881968| 18815.73022987926|3110.127319623236|  -271.8078|  36.52871|462.94504| 6.503996E-4| 29|  9|  1|
| 59989.8680687528|   18511.943681437|3289.819086739001|  58.250034| 109.10752|301.58124| 5.603535E-4| 29|  9|  1|
+-----------------+------------------+-----------------+-----------+----------+---------+------------+---+---+---+
only showing top 4 rows

CPU times: user 5.05 ms, sys: 3.13 ms, total: 8.18 ms
W

In [12]:
%%time
# read suhalo data and save in subdf
subhalofile = 'hdfs://sohnic:54310/data/TNG300/snap99/subhalo.parquet.snappy'
subdf = spark.read.option("header","true").option("recursiveFileLookup","true").parquet(subhalofile)
subdf.show(4)

# change the column names to avoid the confunsion in comparision with ptldf.
subdf = subdf.withColumnRenamed("px", "subhalo_px")
subdf = subdf.withColumnRenamed("py", "subhalo_py")
subdf = subdf.withColumnRenamed("pz", "subhalo_pz")
subdf = subdf.withColumnRenamed("vx", "subhalo_vx")
subdf = subdf.withColumnRenamed("vy", "subhalo_vy")
subdf = subdf.withColumnRenamed("vz", "subhalo_vz")
subdf = subdf.withColumnRenamed("StarHalfMass", "subhalo_StarHalfMass")
subdf = subdf.withColumnRenamed("sub_id", "subhalo_id")
# .withColumnRenamed("column_name1", "column_name2"): replace the name of the "column_name1" with "column_name2"

subdf.show(4)

+---------+---------+----------+----------+----------+---------+------------+-------+
|       px|       py|        pz|        vx|        vy|       vz|StarHalfMass| sub_id|
+---------+---------+----------+----------+----------+---------+------------+-------+
|23283.885|112780.58| 126981.93|  432.6933|-199.78764|362.10175|0.0017790699|2760000|
|23324.938|112736.08|126996.336| 444.65942| -177.0174| 320.3663|         0.0|2760001|
| 9236.999|39951.387|  60465.26|-96.375465|-0.3727351| 96.24267|0.0048540053|2760002|
|23467.604|  63549.3| 47748.617|  93.20628| 122.15136|41.819004| 8.944091E-4|2760003|
+---------+---------+----------+----------+----------+---------+------------+-------+
only showing top 4 rows

+----------+----------+----------+----------+----------+----------+--------------------+----------+
|subhalo_px|subhalo_py|subhalo_pz|subhalo_vx|subhalo_vy|subhalo_vz|subhalo_StarHalfMass|subhalo_id|
+----------+----------+----------+----------+----------+----------+--------------------

We will use subhalos with the stellar mass within a two stellar half mass radius larger than 10^9 M_sun. Thus, we fill the subdf based on the "StarHalfMass" column.

In [7]:
subdf = subdf.filter(F.col("StarHalfMass")/h>0.1)
# .filter(A): filter a Saprk Data Frame based on the condition A.
# F.col(X): refering the data of a column X from a Spark Data Frame
# note that the the unit of "StarHalfMass" data is M_sun/h

# 2. Saving the particle data within the 50kpc spheres centered at each subhalo
Suppose we have four particles (ptl1, ptl2, ptl3, ptl4) in the ptldf and tow subhalos (sub1, sub2) in the subdf. If ptl1 and ptl2 are closer to sub1 than 50 kpc, ptl3 is closer to sub2 than 50 kpc, and ptl5 is closer to both sub1 and sub2 than 50 kpc, the resulting table will be

|particle data|subhalo data|
|---|---|
|ptl1 data | sub1 data|
|ptl2 data | sub1 data|
|ptl3 data | sub2 data|
|ptl4 data | sub1 data|
|ptl4 data | sub2 data|.

Specifically, we will construct the above-like table by 'joining' ptldf and 'subdf'. 

## 2.1. Assign subbox number to each particle and subhalo

We first divide the simulation box into small subboxes and identify particles and subhalos belong to each small box (assign box numbers to each particle and subhalo.

In [8]:
# define some constants 
id_size = 6000 # number of subboxes
subbox_size = box_size/id_size # the size of subboxes
half_box = box_size/2 # half subbox size

In [9]:
# assign box number to each particle
ptldf = ptldf.withColumn("ix", floor(F.col("px") / subbox_size ).cast('int'))
ptldf = ptldf.withColumn("iy", floor(F.col("py") / subbox_size ).cast('int'))
ptldf = ptldf.withColumn("iz", floor(F.col("pz") / subbox_size ).cast('int'))

# .withColumn("a", x): replace the values of a column "a" with x. If the column "a" does not exist, add the column named "a" and fill its value with x.
# .cast(X): converts the datatype into X.

In [13]:
# assign box number to each subhalo
subdf = subdf.withColumn("subhalo_ix", floor(F.col("subhalo_px") / subbox_size).cast('int'))
subdf = subdf.withColumn("subhalo_iy", floor(F.col("subhalo_py") / subbox_size).cast('int'))
subdf = subdf.withColumn("subhalo_iz", floor(F.col("subhalo_pz") / subbox_size).cast('int'))

## 2.2. Broadcast subhalo data

ptldf and subdf are saved separted into several working nodes. Because we have to check the distances from every particle to the all subhalos, all working nodes should have the entire subdf data. We share the enteir subdf with all working nodes by following code:

In [14]:
broadcast_subdf = F.broadcast(subdf)

## 2.3. 'join' based on the subbox numbers

Calculating distances from all particles to all subhalos requires very long computation time.
To reduce the computation, we first divides the simulation box into small boxes and join the particles with the subhalos in the box where the particles are and the boxes next to it. 

In [15]:
# Function to determine if the difference between a particle's boxnumber and a subhalo's boxnumber is smaller than or equal to one.
def int_ptl2subhalo(ptl_boxnumber, subhalo_boxnumber):
    id_diff = F.least(F.abs(F.col(ptl_boxnumber)-F.col(subhalo_boxnumber)), id_size-F.abs(F.col(ptl_boxnumber)-F.col(subhalo_boxnumber)))
    return id_diff <= 1
# F.abs(F.col(ptl_boxnumber)-F.col(subhalo_boxnumber)): difference between particle and subhalo boxnumber
# id_size-F.abs(F.col(ptl_boxnumber)-F.col(subhalo_boxnumber)): difference of boxnumber taking the simulation boundary of boxnubmer
    
# spark Data Frame joined based on boxnumber
boxnumber_joined_df = (
    ptldf.alias("particles").join(
        broadcast_subdf.alias("subhalos"),
        (int_ptl2subhalo("particles.ix", "subhalos.subhalo_ix")  &
         int_ptl2subhalo("particles.iy", "subhalos.subhalo_iy")  &
         int_ptl2subhalo("particles.iz", "subhalos.subhalo_iz"))
        )
)
# df.alias(X): alias (refer to) data frame df with X.
# df1.join(df2, condition): join df1 with df2 if condition is satisfied.

boxnumber_joined_df.show(4)

+-----------------+-----------------+-----------------+---------+---------+---------+-----------+----+---+---+----------+----------+----------+----------+----------+----------+--------------------+----------+----------+----------+----------+
|               px|               py|               pz|       vx|       vy|       vz|       mass|  ix| iy| iz|subhalo_px|subhalo_py|subhalo_pz|subhalo_vx|subhalo_vy|subhalo_vz|subhalo_StarHalfMass|subhalo_id|subhalo_ix|subhalo_iy|subhalo_iz|
+-----------------+-----------------+-----------------+---------+---------+---------+-----------+----+---+---+----------+----------+----------+----------+----------+----------+--------------------+----------+----------+----------+----------+
|59733.16062881968|18815.73022987926|3110.127319623236|-271.8078| 36.52871|462.94504|6.503996E-4|1748|550| 91|  59689.17| 18819.373| 3144.3225| -313.7507| -55.04246|  536.9568|                 0.0|    743320|      1747|       550|        92|
|59733.16062881968|18815.7302298

## 2.4. Calculate the squared distance of the particles from the subhalo positions.

In [18]:
# position difference between particles and subhalos
boxnumber_joined_df = boxnumber_joined_df.withColumn("rel_px", F.pmod(F.col("particles.px") - F.col("subhalos.subhalo_px") + half_box, box_size) - half_box)
boxnumber_joined_df = boxnumber_joined_df.withColumn("rel_py", F.pmod(F.col("particles.py") - F.col("subhalos.subhalo_py") + half_box, box_size) - half_box)
boxnumber_joined_df = boxnumber_joined_df.withColumn("rel_pz", F.pmod(F.col("particles.pz") - F.col("subhalos.subhalo_pz") + half_box, box_size) - half_box)

# squared distances of the particles from the subhalos
boxnumber_joined_df = boxnumber_joined_df.withColumn("sq_dist_subhalo2ptl", F.pow(F.col("rel_px"), 2) + F.pow(F.col("rel_py"), 2) + F.pow(F.col("rel_pz"), 2))

## 2.5. Construct and save a joined data frame filtered by $d_{\rm ptl-subhalo}<50 {\rm ~kpc}$

In [25]:
sq_radius = (50*h)**2 # 50kpc aperture size # note we position in our data frames has kpc/h unit
joined_df = boxnumber_joined_df.filter(F.col("sq_dist_subhalo2ptl")<sq_radius)

# joined_df.show(4)

In [ ]:
# connect particle data to subhalo catalog
subcat = pd.read_csv("subhalocat300.txt", sep=' ')
subcat_type =  T.StructType([T.StructField('subhalo_id',T.IntegerType(), True),
                             T.StructField('StarHalfRad',T.IntegerType(), True)
                      ])
subcat = spark.createDataFrame(subcat[["SubfindID", "StarHalfRad"]], subcat_type)
joined_df = joined_df.join(subcat, "subhalo_id")

In [ ]:
# save the joined_df
# It takes several hours. I recommend to do this task in .py file and run in the terminal.
filename = 'hdfs://sohnic:54310/data/TNG300/snap99/subhalos2ptls.parquet.snappy'
joined_df.write.option("compression", "snappy").mode("overwrite").partitionBy("subhalo_id").parquet(filename)
